In [92]:
import csv
from datetime import datetime
import pandas as pd
import requests
import time

def get_yahoo_finance_intraday(symbol, dt, interval='1m'):
    """
    Retrieve intraday stock data from Yahoo Finance/data directory. 
    Parameters
    ----------
    symbol : str
        Company symbol.
    interval : string
        Interval between stock values(5m| 1m | 1h).
    dt : str
        date (yyyy-mm-dd).
    Returns
    -------
    df : pandas.DataFrame
        DataFrame containing the opening price, high price, low price,
        closing price, and volume. The index contains the times associated with
        the retrieved price values.
    """
    
    #convert start and date to unix timestamp 
    try: 
        start_dt = datetime.strptime(dt, '%Y-%m-%d')
        starttime = int(time.mktime(start_dt.timetuple()))
        #adding 3600s to starttime to get EOD time
        endtime = starttime + 24 * 60 * 60
    except:
        raise('Date format is invalid')
    destination = os.path.join(os.pardir,
                                   'data/{symbol}_{filename}_{interval}.csv'.format(
                                    filename=dt,
                                    interval=interval,
                                    symbol=symbol
                                    ))
    
    # we don't query if data exists 
    if os.path.isfile(destination):
        result = pd.read_csv(destination)
        result = result.set_index('timestamp')
        return result
    
    uri = 'https://query1.finance.yahoo.com/v8/finance/chart' \
          '/{page}?symbol={symbol}&period1={start}&period2={end}' \
          '&interval={interval}&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US' \
          '&region=US&crumb=sJVUbTOFjBq&corsDomain=finance.yahoo.com'.format(page=symbol,
                                                                             symbol=symbol,
                                                                             start=starttime ,
                                                                             end=endtime,
                                                                             interval=interval)
   
    req = requests.get(url = uri) 
    #extracting response text  
    data = req.json() 
    if data['chart']['error']:
        raise Exception(data['chart']['error']['description'])
    result = data['chart']['result']
    result_df = None
    if result:
        result = pd.DataFrame.from_dict({
                'timestamp': result[0]['timestamp'],
                'open': result[0]['indicators']['quote'][0]['open'],
                'close': result[0]['indicators']['quote'][0]['close'],
                'high': result[0]['indicators']['quote'][0]['high'],
                'low': result[0]['indicators']['quote'][0]['low'],
                'volume': result[0]['indicators']['quote'][0]['volume']
            })
        result = result.set_index('timestamp')
        #save result to data directory
        result['symbol'] = symbol
        result.to_csv(destination)
    return result

In [91]:
djia_components = ['MMM', 'AXP', 'AAPL', 'CAT', 'CVX', 'CSCO',
                   'DWDP', 'XOM', 'INTC', 'IBM', 'JNJ', 'JPM', 'MCD',
                   'MRK', 'MSFT', 'NKE', 'PFE', 'BA', 'KO', 'GS', 'HD',
                   'PG', 'TRV', 'DIS', 'UTX', 'UNH', 'VZ', 'V', 'WBA', 'WMT']

dt_list = [ '2019-05-23', '2019-05-24', '2019-05-28', '2019-05-29']
for comp in djia_components:
    for dt in dt_list:
        get_yahoo_finance_intraday(comp, dt)